In [ ]:
//---- imports for HIPO4 library
import org.jlab.jnp.hipo4.io.*;
import org.jlab.jnp.hipo4.data.*;
//---- imports for GROOT library
import org.jlab.groot.data.*;
import org.jlab.groot.graphics.*;
//---- imports for PHYSICS library
import org.jlab.jnp.physics.*;
import org.jlab.jnp.reader.*;


HipoReader reader = new HipoReader(); // Create a reader obejct
reader.open("/home/justind/DATA/out_6489_2xx_3xx.hipo"); // open a file

In [ ]:
public class DvcsEvent {
    LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.2,10.2);
    LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,1.878);
    LorentzVector  velectron = new LorentzVector();
    LorentzVector  vphoton = new LorentzVector();
    LorentzVector  vhadron = new LorentzVector();
    double MNUC=1.878;
    
    public DvcsEvent() {
      // This constructor no parameter.
      System.out.println("setting the default DVCS event for hadron :" + MNUC );
   }
    public DvcsEvent(double mass) {
      // This constructor no parameter.
      MNUC=mass;
      System.out.println("setting the default DVCS event for hadron :" + MNUC );
   }

   public void setElectron(Bank particles, int ne) {
      velectron.setPxPyPzM(particles.getFloat("px",ne),
                                 particles.getFloat("py",ne),
                                 particles.getFloat("pz",ne),
                                 0.0005);
   }

   public void setPhoton(Bank particles, int ng) {
      vphoton.setPxPyPzM( particles.getFloat("px",ng),
                                particles.getFloat("py",ng),
                                particles.getFloat("pz",ng),
                                0.0);
   }
   public void setHadron(Bank particles, int nh) {
      vhadron.setPxPyPzM(particles.getFloat("px",nh),
                                particles.getFloat("py",nh),
                                particles.getFloat("pz",nh),
                                this.MNUC);
   }
   public LorentzVector W(){
       LorentzVector tmp = new LorentzVector();
       tmp.copy(vBeam);
       tmp.add(vTarget).sub(velectron);
       return tmp;
       
   }
   public LorentzVector Q2(){
       LorentzVector  tmp = new LorentzVector();
       tmp.copy(vBeam);
       tmp.sub(velectron);
       return tmp;
       
   }
   public LorentzVector DVCSmissX(){
       LorentzVector  tmp = new LorentzVector();
       tmp.copy(vBeam);
       tmp.add(vTarget);
       tmp.sub(velectron);
       tmp.sub(vphoton);
       return tmp;
   }
   public LorentzVector ehehgX(){
       LorentzVector  tmp = new LorentzVector();
       tmp.copy(vBeam);
       tmp.add(vTarget);
       tmp.sub(velectron);
       tmp.sub(vphoton);
       tmp.sub(vhadron);
       return tmp;
   }
   public LorentzVector ehehX(){
       LorentzVector  tmp = new LorentzVector();
       tmp.copy(vBeam);
       tmp.add(vTarget);
       tmp.sub(velectron);
       tmp.sub(vhadron);
       return tmp;
   }
   public double MM2(){
       return this.DVCSmissX().mass2();
   }
   public double Mpx(){
       return this.ehehgX().px();
   }
   public double Mpy(){
       return this.ehehgX().py();
   }
   public double Mpz(){
       return this.ehehgX().pz();
   }
   public boolean DVCScut(){
       boolean cut=(-this.Q2().mass2()>1 && this.W().mass()>2 && this.vphoton.e()>1);
       return cut;
   }
   public double Xb(){
       return (-this.Q2().mass2())/(2*0.938*(this.vBeam.p()-this.velectron.p()));
   }
   public double DTheta(){
        LorentzVector tmp = new LorentzVector();
        tmp.copy(this.ehehX());
    return Math.toDegrees(vphoton.vect().angle(tmp.vect()));
   }
   public double DPhi(){
        LorentzVector temp = new LorentzVector();
        temp.copy(this.ehehX());
    return Math.toDegrees(temp.vect().phi() - vphoton.vect().phi());
   }
   public double PhiPlane(){
       double Phi;
       Vector3 leptonicPlane = new Vector3();
       Vector3 hadronicPlane = new Vector3();
       leptonicPlane.copy(vBeam.vect().cross(velectron.vect()));
       hadronicPlane.copy(vhadron.vect().cross(vphoton.vect()));
       Phi = Math.toDegrees(leptonicPlane.angle(hadronicPlane));
       return Phi;
   }
    public double deltaPhiPlane(){
            double deltaphiplane;
            LorentzVector Q = new LorentzVector();
            Q.copy(vBeam);
            Q.sub(velectron);
            Vector3 norm_Pro_VPho = (vhadron.vect().cross(Q.vect()));
            Vector3 norm_Pro_Pho = (vhadron.vect().cross(vphoton.vect()));                 
            deltaphiplane = Math.toDegrees(norm_Pro_Pho.angle(norm_Pro_VPho));
            if(norm_Pro_VPho.dot(vphoton.vect()) < 0 ) deltaphiplane = -1*deltaphiplane;
        return deltaphiplane;
    }
    public double coneangle(){
        LorentzVector temp = new LorentzVector();
        temp.copy(this.ehehX());
        return Math.toDegrees(this.vphoton.vect().angle(temp.vect()));
    }
}

In [ ]:
Event     event = new Event();
Bank  particles = new Bank(reader.getSchemaFactory().getSchema("REC::Particle"));
Bank  run      = new Bank(reader.getSchemaFactory().getSchema("REC:Event"));
DvcsEvent ev = new DvcsEvent();

In [ ]:
reader.getEvent(event,0); //Reads the first event and resets to the begining of the file
int times=0;
//LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.2,10.2);
//LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,1.878);
//LorentzVector  velectron = new LorentzVector();
LorentzVector  vtmp = new LorentzVector();
LorentzVector  vtemp = new LorentzVector();
//LorentzVector  vproton = new LorentzVector();
//LorentzVector  vphoton = new LorentzVector();
//LorentzVector  vdeuteron = new LorentzVector();
LorentzVector  vW = new LorentzVector();
LorentzVector  vQ2 = new LorentzVector();
LorentzVector  vMMass = new LorentzVector();
LorentzVector  vMMom = new LorentzVector();
Particle  proton = new Particle();
Particle neutron = new Particle();
Particle deuteron = new Particle();
Particle electron = new Particle();
Particle photon = new Particle();
double Xbj;
double el_en_max=0;
double ph_en_max=0;
double d_en_max=0;
int nelec=0;
int ndvcs=0;
int nphot=0;
int ndeut=0;
int ne=-1;
int ng=-1;
int nd=-1;
//double MNUC=1.878;
int PIDNUC=45;
int counter=0;

In [ ]:
// histos
H1F W = new H1F("W" ,100, 0, 10.0);
W.setTitleX("W [GeV]");
H1F Q2 = new H1F("Q2",100, 0.1, 4.0);
Q2.setTitleX("Q^2 [GeV/c^2]");
H1F MMass = new H1F("MMass",100,-5,40);
MMass.setTitleX("Missing Mass Squared");
H1F MMom = new H1F("MMass",100,-30,30);
MMom.setTitleX("Missing Momentum");
H1F MMomx = new H1F("MMass",100,-10,10);
MMomx.setTitleX("Missing X Momentum");
H1F MMomy = new H1F("MMass",100,-10,10);
MMomy.setTitleX("Missing Y Momentum");
H1F MMomz = new H1F("MMass",100,-10,15);
MMomz.setTitleX("Missing Z Momentum");
H2F WvsQ2 = new H2F("W vs Q2", "W vs Q2", 100,0,7,100,0,10);
WvsQ2.setTitleX("W [GeV]");
WvsQ2.setTitleY("Q^2 [GeV/c^2]");
H2F ThvsPhi = new H2F("Theta vs Phi","Theta vs Phi",100,-180,180,100,0,180);
ThvsPhi.setTitleX("Phi [Degrees]");
ThvsPhi.setTitleY("Theta [Degrees]");
H2F Q2vsXbj = new H2F("Q2 vs Xbj","Q2 vs Xbj",100,0,10,100,0,1);
Q2vsXbj.setTitleX("Q^2 [GeV/c^2]");
Q2vsXbj.setTitleY("Xbj");
H2F MMvsMpz = new H2F("Q2 vs Xbj","Q2 vs Xbj",100,-2,2,100,-10,10);
MMvsMpz.setTitleX("Missing Mass");  
MMvsMpz.setTitleY("Missing Z Momentum");
H2F MpxvsMpz = new H2F("Q2 vs Xbj","Q2 vs Xbj",100,-2,2,100,-10,10);
MpxvsMpz.setTitleX("Missing X Momentum");
MpxvsMpz.setTitleY("Missing Z Momentum");
H1F ThetaHist = new H1F("ThetaHist",100,0,50);
ThetaHist.setTitle("Photon Theta");
H1F DThetaHist = new H1F("DThetaHist",100,0,50);
DThetaHist.setTitle("DTheta");
H1F MissThetaHist = new H1F("MissThetaHist",100,0,180);
MissThetaHist.setTitle("Missing Photon Theta");
H1F DPhiHist = new H1F("DPhiHist",100,-180,180);
DPhiHist.setTitle("DPhi");
H1F PhiPlaneHist = new H1F("PhiPlaneHist",100,0,180);
PhiPlaneHist.setTitle("Photon Phi Plane");
H1F DeltaPhiPlaneHist = new H1F("DeltaPhiPlane",100,0,180);
DeltaPhiPlaneHist.setTitle("Delta Phi Plane");
H1F ConeAngleHist = new H1F("ConeAngleHist",100,-15,180);
ConeAngleHist.setTitle("Cone Angle");

H1F edgXmissingE = new H1F("edgXmissingE",100,-30,30);
edgXmissingE.setTitle("eDGammaX Missing Energy");
H1F edgXmissingM2 = new H1F("edgXmissingM2",100,-30,30);
edgXmissingM2.setTitle("eDGammaX Missing Mass2");
H1F edgXmissingP = new H1F("edgXmissingP",100,0,30);
edgXmissingP.setTitle("eDGammaX Missing Momentum");
H1F edXmissingM2 = new H1F("edXmissingM2",100,-30,30);
edXmissingM2.setTitle("eDX Missing Mass2");
H1F egXmissingM2 = new H1F("egXmissingM2",100,-30,30);
egXmissingM2.setTitle("eGammaX Missing Mass2");

In [ ]:
//with loops
while(reader.hasNext()==true){
    reader.nextEvent(event);
    event.read(particles);
    el_en_max=0;
    ph_en_max=0;
    d_en_max=0;
    nelec=0;
    ndeut=0;
    nphot=0;
    if(particles.getRows()>3){
        for(int npart=0; npart<particles.getRows(); npart++){
        int pid = particles.getInt("pid", npart);
        if(pid==11){ 
            nelec++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 0.0005);
            if(vtmp.e()>el_en_max)ne=npart;
        }
        else if(pid==22){
            nphot++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 0.0);
            if(vtmp.e()>ph_en_max)ng=npart;
            
        }
        else if(pid==PIDNUC){
            ndeut++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 ev.MNUC);
            if(vtmp.e()>d_en_max)nd=npart;
        }   
    }
    if(ndeut>=1 && nelec>=1 && nphot>=1){
        ev.setElectron(particles,ne);
        ev.setPhoton(particles,ng);
        ev.setHadron(particles,nd);
    
        vW.copy(ev.W());
        vQ2.copy(ev.Q2());
    
        PhiPlaneHist.fill(ev.PhiPlane());
        DeltaPhiPlaneHist.fill(ev.deltaPhiPlane());
            
        DPhiHist.fill(ev.DPhi());
        
        //ConeAngleHist.fill(ev.coneangle());
        
        if(ev.DVCScut()){
            ndvcs++;
            ThvsPhi.fill(Math.toDegrees(ev.vhadron.phi()),Math.toDegrees(ev.vhadron.theta()));
            Xbj=ev.Xb();
            Q2vsXbj.fill(-vQ2.mass2(),Xbj); 
            
            ThetaHist.fill(Math.toDegrees(ev.vphoton.theta()));
            DThetaHist.fill(ev.DTheta());
            vtemp.copy(ev.ehehX());
            MissThetaHist.fill(Math.toDegrees(vtemp.theta()));
            
            ConeAngleHist.fill(ev.coneangle());

            W.fill(vW.mass());
            Q2.fill(-vQ2.mass2());
            WvsQ2.fill(vW.mass(),-vQ2.mass2());
            
            MMass.fill(ev.MM2());
            
            edgXmissingE.fill(ev.ehehgX().e());
            edgXmissingM2.fill(ev.ehehgX().mass2());
            edgXmissingP.fill(ev.ehehgX().p());
            edXmissingM2.fill(ev.ehehX().mass2());
            egXmissingM2.fill(ev.DVCSmissX().mass2());
            
            
            
            if((Math.toDegrees(ev.vphoton.theta())<5)  && ev.coneangle()<3 && ev.MM2()<15){
            counter++;
            MMomx.fill(ev.Mpx());
            MMomy.fill(ev.Mpy());
            MMomz.fill(ev.Mpz());

                }
            }
        }
    }
}
    
System.out.println("total dvcs events: " + ndvcs);
System.out.println("total filtered events: " + counter);


In [ ]:

EmbeddedCanvas ec = new EmbeddedCanvas(1000,1000);
/*ec.divide(3,4);
//ec.getPad(0).getAxisZ().setLog(true);
ec.cd(0).draw(WvsQ2);
//ec.getPad(1).getAxisZ().setLog(true);
ec.cd(1).draw(Q2vsXbj);
ec.cd(2).draw(W);
ec.cd(3).draw(Q2);
ec.cd(4).draw(ThvsPhi);*/
//ec.cd(5).draw(MMass);
//ec.cd(6).draw(MMomx);
//ec.cd(7).draw(MMomy);
ec.cd(8).draw(MMomz);

ec.getScreenShot();


In [ ]:
EmbeddedCanvas ec2 = new EmbeddedCanvas(1200,600);
ec2.divide(3,1);
ec2.cd(0).draw(ThetaHist);
ec2.cd(1).draw(MissThetaHist);
ec2.cd(2).draw(DThetaHist);
ec2.getScreenShot();

In [ ]:
EmbeddedCanvas ec3 = new EmbeddedCanvas(1200,1200);
//ec3.divide(2,2);
//ec3.cd(0).draw(DPhiHist);
//ec3.cd(1).draw(PhiPlaneHist);
//ec3.cd(2).draw(DeltaPhiPlaneHist);
ec3.cd(3).draw(ConeAngleHist);
ec3.getScreenShot();

In [ ]:
EmbeddedCanvas ec4 = new EmbeddedCanvas(1500,1500);
ec4.divide(3,3);
//ec4.cd(0).draw(DPhiHist);
ec4.cd(1).draw(edgXmissingE);
ec4.cd(2).draw(edgXmissingM2);
ec4.cd(3).draw(edgXmissingP);
ec4.cd(4).draw(edXmissingM2);
ec4.cd(5).draw(egXmissingM2);
ec4.cd(6).draw(ConeAngleHist);
ec4.cd(7).draw(MMomx);
ec4.cd(8).draw(MMomy);
ec4.getScreenShot();